In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import numpy as np
import os

In [ ]:
# choose a month
month = '07'

# load in data sets
# the fields all have dimensions time, pressure level, latitude and longitude.
# (the time dimension (t-1h, t, t+1h) is only used for derivatives in the scale analysis
# and ignored elsewhere, i.e. just the central time step is used)

script_dir = os.path.dirname(__name__)
rel_path = f'files/2020_{month}_01_hourly.nc'
abs_file_path = os.path.join(script_dir, rel_path)
rel_path_surf = f'files/2020_{month}_01_hourly_surf.nc'
abs_file_path_surf = os.path.join(script_dir, rel_path_surf)

data = xr.open_dataset(abs_file_path)
uWind = data['u']
vWind = data['v']
wWind = data['w']
temp = data['t']
div = data['d']
geop = data['z']
vort = data['vo']
surf_press = xr.open_dataset(abs_file_path_surf)['sp']
pressure_levels = np.array(data.level)

lon = data.longitude
lat = data.latitude
grav = 9.80665
geop_height = geop / grav

In [ ]:
# The fields are set to nan where there are mountains/ surface elevations
# that are higher than the respective pressure level

def orography(field):
    for i in range(len(pressure_levels)):
        field[:,i,:,:] = np.where(surf_press<pressure_levels[i]*100,np.nan,field[:,i,:,:])
    return field

[uWind,vWind,wWind,temp,geop,div,vort] = [orography(field) for field in [uWind,vWind,wWind,temp,geop,div,vort]]

In [ ]:
# store fields and grid so they can be imported into other notebooks

fields = [uWind,vWind,wWind,temp,geop,div,vort,geop_height]
%store fields
grid = [lon,lat,pressure_levels]
%store grid
%store month

In [ ]:
# for calculating the gradient of a given field, first the distance between
# grid points is converted to metres. the distance in meridional direction is
# constant (deg_to_m) but varies in zonal direction depending on the latitude (dist)

deg_to_m = 111120

dist = np.array(deg_to_m * np.cos(np.deg2rad(lat)))
dist = np.swapaxes(np.tile(dist,[len(lon),1]),0,1)

def grad(field):
    '''this function calculates the gradient of field'''
    grad = np.zeros((2,*field.isel(time=0).shape))
    
    for j in range(1,len(field.latitude)-1):
        grad[1,:,j,:] = - 1 / (2 * deg_to_m) * (field.isel(time=1)[:,j+1,:] - field.isel(time=1)[:,j-1,:])
    grad[1,:,0,:] = - 1 / deg_to_m * (field.isel(time=1)[:,1,:] - field.isel(time=1)[:,0,:])
    grad[1,:,-1,:] = - 1 / deg_to_m * (field.isel(time=1)[:,-1,:] - field.isel(time=1)[:,-2,:])
    
    for i in range(1,len(field.longitude)-1):
        grad[0,:,:,i] = 1 / (2 * dist[:,i]) * (field.isel(time=1)[:,:,i+1] - field.isel(time=1)[:,:,i-1])
    grad[0,:,:,0] = 1 / dist[:,0] * (field.isel(time=1)[:,:,1] - field.isel(time=1)[:,:,0])
    grad[0,:,:,-1] = 1 / dist[:,-1] * (field.isel(time=1)[:,:,-1] - field.isel(time=1)[:,:,-2])
    
    return grad

In [ ]:
def divergence(field_x, field_y):
    '''calculates the divergence of field = (field_x,field_y)'''
    div_field = xr.zeros_like(field_x)
    div_field[1,:,:,:] = (grad(field_x)[0,:,:,:] + grad(field_y)[1,:,:,:])
    div_field.attrs['long_name'] = f'divergence of wind'
    div_field.attrs['units'] = f'm/s /m'
    return div_field

In [ ]:
plt.figure(figsize=(15,6))
plt.pcolormesh((divergence(uWind,vWind)-div)[1,0,10:-10,:])
plt.colorbar()

In [ ]:
def plot(field,N=90,S=90,W=0,E=360,pressure_level=0,vmin=None,vmax=None):
    '''this function plots field with an areal extend of [N,S,W,E] at pressure_level.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       the geopotential height is displayed as contour lines.
       vmin and vmax are mainly thought to be used for divergence and vorticity
       of the wind field and can be ignored for other fields.
       the geopotential height is displayed as contour lines (in m).
    '''
    N = 90-N
    S = 90+S
    
    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[1,pressure_level,N:S,W:E],
                    cmap='viridis', levels=50, vmin=vmin, vmax=vmax)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')

    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat), label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at pressure level {pressure_levels[pressure_level]} hPa", fontsize=15)
    fig.tight_layout()

In [ ]:
def plot_wind(field,N=90,S=90,W=0,E=360,pressure_level=0,spacing=5,vmin=None,vmax=None):
    '''this function plots field with an areal extend of [N,S,W,E] at pressure_level
       with the wind field displayed as arrows.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       spacing is the space inbetween arrows in degrees.
       the geopotential height is displayed as contour lines.
       vmin and vmax are mainly thought to be used for divergence and vorticity
       of the wind field and can be ignored for other fields.
       the geopotential height is displayed as contour lines (in m).
    '''
    N = 90-N
    S = 90+S
    
    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[1,pressure_level,N:S,W:E],
                    cmap='viridis', levels=50, vmin=vmin, vmax=vmax)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')
    
    Q = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                uWind[1,pressure_level,N:S,W:E][::spacing,::spacing],
                vWind[1,pressure_level,N:S,W:E][::spacing,::spacing])
    Qk = ax.quiverkey(Q,0.5,-0.1,np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]),
            label="{:.0f}".format(np.array(np.nanmax(uWind[1,pressure_level,N:S,W:E][::spacing,::spacing]))) + " m/s wind velocity",
            labelpos = 'E')
    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat),label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at pressure level {pressure_levels[pressure_level]} hPa", fontsize=15)
    fig.tight_layout()

In [ ]:
def plot_grad(field,N=90,S=90,W=0,E=360,pressure_level=0,spacing=5,vmin=None,vmax=None):
    '''this function plots field with an areal extend of [N,S,W,E] at pressure_level
       with the gradient of field displayed as arrows.
       W is given in degrees east and has to be smaller than E,
       also no negative values are allowed.
       spacing is the space inbetween arrows in degrees.
       the geopotential height is displayed as contour lines.
       vmin and vmax are mainly thought to be used for divergence and vorticity
       of the wind field and can be ignored for other fields.
       the geopotential height is displayed as contour lines (in m).
    '''
    N = 90-N
    S = 90+S
    
    fig, ax = plt.subplots(figsize=(15,8), subplot_kw={'projection': ccrs.PlateCarree()})
    im = ax.contourf(lon[W:E], lat[N:S], field[1,pressure_level,N:S,W:E],
                    cmap='viridis', levels=50, vmin=vmin, vmax=vmax)
    
    im2 = ax.contour(lon[W:E], lat[N:S], geop_height[1,pressure_level,N:S,W:E])
    ax.clabel(im2, im2.levels, inline=True,colors='k')
    
    Q = ax.quiver(lon[W:E][::spacing], lat[N:S][::spacing],
                grad(field)[0,pressure_level,N:S,W:E][::spacing,::spacing],
                grad(field)[1,pressure_level,N:S,W:E][::spacing,::spacing])
    Qk = ax.quiverkey(Q,0.5,-0.1,np.nanmax(grad(field)[:,pressure_level,N:S,W:E][::spacing,::spacing]),
                      label="{:.0f}".format(100000 * np.nanmax(grad(field)[:,pressure_level,N:S,W:E][::spacing,::spacing]))
                           +f" {field.units} / 100 km {field.long_name} gradient",labelpos = 'E')
    ax.add_feature(cf.COASTLINE)
    ax.add_feature(cf.BORDERS)
    ax.set_xticks([0],[0])
    ax.set_yticks([0],[0])

    col = fig.colorbar(im, orientation='horizontal', fraction=0.039*len(lon)/len(lat),label=f"{field.long_name} [{field.units}]")
    ax.set_title(f"{field.long_name} at pressure level {pressure_levels[pressure_level]} hPa", fontsize=15)
    fig.tight_layout()

### Use the functions to display the fields alone (plot), with their gradient (plot_grad), or with the wind (plot_wind)

try out different regions and altitudes

In [ ]:
pressure_levels # in hPa

In [ ]:
plot(temp,pressure_level=-2)

In [ ]:
plot(temp,N=60,S=-40,W=0,E=20,pressure_level=2)

In [ ]:
plot_grad(temp,pressure_level=3)

In [ ]:
plot_wind(temp,N=20,S=90,W=80,E=180,pressure_level=2,spacing=4)

For displaying the divergence and vorticity it is useful to set the parameters vmin and vmax. Different values may be appropiate for different areas and pressure levels.

In [ ]:
plot_wind(div,N=60,S=30,W=280,E=360,pressure_level=2,spacing=3,vmin=-0.0001,vmax=0.0001)

In [ ]:
plot_wind(vort,N=60,S=30,W=280,E=360,pressure_level=3,spacing=3,vmin=-0.0001,vmax=0.0001)